In [ ]:
import numpy as np
import pandas as pd
import urllib
import urllib.parse
import os
import requests
import glob
import time

# create a table with the following fields:
#   - account name
#   - name of the resort
#   - state the resort is located in
#   - address of the resort
#   - zip code of the resort
#   - minimum elevation (in meters) of the resort
#   - maximum elevation (in meters) of the resort
#   - time difference (compared to UTC)

# Silverton is excluded from the analysis because it's not present in the ZRankings
# New_Mexico is spelled as it is so it can easily be merged to the weather data in a later stage

accounts = [['resortalyeska', 'Alyeska', 'Alaska', '1000 Arlberg Ave, Girdwood, AK 99587, USA', '99587'],
            ['skiheavenly', 'Heavenly', 'California', '4080 Lake Tahoe Blvd, South Lake Tahoe, CA 96150, USA', '96150'],
            ['kirkwoodmtn', 'Kirkwood', 'California', '1501 Kirkwood Meadows Dr, Kirkwood, CA 95646, USA', '95646'],
            ['mammothmountain', 'Mammoth Mountain', 'California', '10001 Minaret Rd, Mammoth Lakes, CA 93546, USA', '93546'],
            ['northstar_california', 'Northstar California', 'California', '5001 Northstar Dr, Truckee, CA 96161, USA', '96161'],
            ['squawalpine', 'Squaw Alpine', 'California', '1960 Squaw Valley Rd, Olympic Valley, CA 96146, USA', '96146'],
            ['sugarbowlresort', 'Sugar Bowl', 'California', '629 Sugar Bowl Rd, Norden, CA 95724, USA', '95724'],
            ['arapahoe_basin', 'Arapahoe Basin', 'Colorado', '28194 US-6, Keystone, CO 80435, USA', '80435'],
            ['aspensnowmass', 'Aspen Snowmass', 'Colorado', '120 Carriage Way, Snowmass Village, CO 81615, USA', '81615'],
            ['aspenco', 'Aspen', 'Colorado', '601 E Dean St, Aspen, CO 81611, USA', '81611'],
            ['beavercreek', 'Beaver Creek', 'Colorado', 'Beaver Creek, Colorado 81620, USA', '81620'],
            ['breckenridgemtn', 'Breckenridge', 'Colorado', '599 Ski Hill Rd, Breckenridge, CO 80424, USA', '80424'],
            ['coppermtn', 'Copper Mountain', 'Colorado', 'Continental Divide Trail, Frisco, CO 80443, USA', '80443'],
            ['skicrestedbutte', 'Crested Butte', 'Colorado', '12 Snowmass Rd, Crested Butte, CO 81225, USA', '81225'],
            ['keystone_resort', 'Keystone', 'Colorado', '1202 Jones Gulch Rd, Keystone, CO 80435, USA', '80435'],
            ['lovelandskiarea', 'Loveland', 'Colorado', 'Dillon, Colorado 80435, USA', '80435'],
            ['skipowderhorn', 'Powderhorn', 'Colorado', '48338 Powderhorn Rd, Mesa, CO 81643, USA', '81643'],
            #['silvertonmtn', 'Silverton', 'Colorado', '6226 State Hwy 110, Silverton, CO 81433, USA', '81433'],
            ['steamboatresort', 'Steamboat', 'Colorado', '2305 Mt Werner Cir, Steamboat Springs, CO 80487, USA', '80487'],
            ['tellurideski', 'Telluride', 'Colorado', '565 Mountain Village Blvd, Telluride, CO 81435, USA', '81435'],
            ['vailmtn', 'Vail', 'Colorado', 'Vail, Colorado 81657, USA', '81657'],
            ['winterparkresort', 'Winter Park', 'Colorado', '85 Parsenn Rd, Winter Park, CO 80482, USA', '80482'],
            ['schweitzer_mountain', 'Schweitzer', 'Idaho', '10,000 Schweitzer Mountain Road, White Pine Lodge, Sandpoint, ID 83864, USA', '83864'],
            ['sunvalley', 'Sun Valley', 'Idaho', 'Kitzbuhler Strasse, Sun Valley, ID 83353, USA', '83353'],
            ['tamarackresort', 'Tamarack', 'Idaho', '311 Village Dr, Tamarack, ID 83615, USA', '83615'],
            ['bigskyresort', 'Big Sky', 'Montana', '50 Big Sky Resort Rd, Big Sky, MT 59716, USA', '59716'],
            ['bridgerbowl', 'Bridger Bowl', 'Montana', '15795 Bridger Canyon Rd, Bozeman, MT 59715, USA', '59715'],
            ['mtroseskitahoe', 'Mount Rose', 'Nevada', '22222 Mt Rose Hwy, Reno, NV 89511, USA', '89511'],
            ['skitaos', 'Taos', 'New_Mexico', 'Taos Ski Valley, New Mexico 87525 USA', '87525'],
            ['mthoodmeadows', 'Mount Hood Meadows', 'Oregon', '14040 Oregon 35, Mt Hood, OR 97041, USA', '97041'],
            ['altaskiarea', 'Alta', 'Utah', '10230 UT-210, Alta, UT 84092, USA', '84092'],
            ['brightonresort', 'Brighton', 'Utah', '8302 S Brighton Loop Rd, Brighton, UT 84121, USA', '84121'],
            ['deervalleyresort', 'Deer Valley', 'Utah', '2250 Deer Valley Dr S, Park City, UT 84060, USA', '84060'],
            ['pcski', 'Park City', 'Utah', '1345 Lowell Ave, Park City, UT 84060, USA', '84060'],
            ['powdermountain', 'Powder Mountain', 'Utah', '6965 E Powder Mountain Rd, Eden, UT 84310, USA', '84310'],
            ['snowbasinresort', 'Snowbasin', 'Utah', '3925 Snow Basin Rd, Huntsville, UT 84317, USA', '84317'],
            ['snowbird', 'Snowbird', 'Utah', '9385 Snowbird Center Trail, Sandy, UT 84092, USA', '84092'],
            ['solitudemtn', 'Solitude', 'Utah', '12000 Big Cottonwood Canyon Rd, Solitude, UT 84121, USA', '84121'],
            ['jaypeakresort', 'Jay Peak', 'Vermont', '830 Jay Peak Rd, Jay, VT 05859, USA', '05859'],
            ['killingtonmtn', 'Killington', 'Vermont', '4763 Killington Rd, Killington, VT 05751, USA', '05751'],
            ['madriverglen', 'Mad River Glen', 'Vermont', ' 57 Schuss Pass, Waitsfield, VT 05673, USA', '05673'],
            ['stowemt', 'Stowe', 'Vermont', '7416 Mountain Rd, Stowe, VT 05672, USA', '05672'],
            ['crystalmountain', 'Crystal Mountain', 'Washington', '33914 Crystal Mountain Blvd, Enumclaw, WA 98022, USA', '98022'],
            ['mtbakerskiarea', 'Mount Baker', 'Washington', 'Mt Baker Hwy, Deming, WA 98244, USA', '98244'],
            ['stevenspass', 'Stevens Pass', 'Washington', 'US-2, Skykomish, WA 98288, USA', '98288'],
            ['grandtargheeresort', 'Grand Targhee', 'Wyoming', '3300 Ski Hill Rd, Alta, WY 83414, USA', '83414'],
            ['jacksonhole', 'Jackson Hole', 'Wyoming', '3395 Cody Ln, Teton Village, WY 83025, USA', '83025'],
            ['sundanceresort', 'Sundance Resort', 'Utah', '8841 Alpine Loop Scenic Byway, Sundance, UT 84604, USA', '84604'],
            ['bogusbasin', 'Bogus Basin', 'Idaho', '2600 Bogus Basin Road, Boise, ID 83702, USA', '83702'],
            ['summitatsnoqualmie', 'The Summit at Snoqualmie', 'Washington', '1001 WA-906, Snoqualmie Pass, WA 98068, USA', '98068'],
            ['skipurg', 'Purgatory', 'Colorado', '1 Skier Pl, Durango, CO 81301, USA', '81301'],
            ['brianheadresort', 'Brian Head', 'Utah', '329 UT-143, Brian Head, UT 84719, USA', '84719'],
            ['sierra_at_tahoe', 'Sierra at Tahoe', 'California', '1111 Sierra-At-Tahoe Rd, Twin Bridges, CA 95735, USA', '95735'],
            ['skieaglecrest', 'Eaglecrest', 'Alaska', '3000 Fish Creek Road, Juneau, AK 99801, USA', '99801'],
            ['silvermountain', 'Silver Mountain', 'Idaho', '914 Sierra Nevada, Kellogg, ID 83837, USA', '83837'],
            ['skisantafe', 'Ski Santa Fe', 'New_Mexico', 'NM-475, Santa Fe, NM 87501, USA', '87501'],
            ['skithebeav', 'Beaver Mountain', 'Utah', '40000 East Highway 89, Garden City, UT 84028, USA', '84028'],
            ['brundagemtn', 'Brundage Mountain', 'Idaho', '3890 Goose Lake Rd, McCall, ID 83638, USA', '83638'],
            ['skieaglepoint', 'Eagle Point', 'Utah', '150 S W Village Cir, Beaver, UT 84713, USA', '84713'],
            ['whitepass', 'White Pass', 'Washington', '48935 US-12, Naches, WA 98937, USA', '98937'],
            ['losttrailskiarea', 'Lost Trail Powder Mountain', 'Montana', ' 9485 US-93, Sula, MT 59871, USA', '59871'],
            ['smugglersnotchvt', 'Smugglers Notch', 'Vermont', '4323 VT-108, Jeffersonville, VT 05464, USA', '05464'],
            ['redlodge', 'Red Lodge Mountain', 'Montana', '305 Ski Run Rd, Red Lodge, MT 59068, USA', '59068'],
            ['monarchmountain', 'Monarch Mountain', 'Colorado', '23715 US-50, Monarch, CO 81227, USA', '81227'],
            ['sugarbush_vt', 'Sugarbush', 'Vermont', '1840 Sugarbush Access Rd, Warren, VT 05674, USA', '05674'],
            ['skiwhitefish', 'Whitefish', 'Montana', '1015 Glades Dr, Whitefish, MT 59937, USA', '59937'],
            ['wolfcreekski', 'Wolf Creek', 'Colorado', 'Top of Wolf Creek Pass, U.S. Hwy. 160 E., Pagosa Springs, CO 81147, USA', '81147'],
            ['mtbachelor', 'Mount Bachelor', 'Oregon', '13000 SW Century Drive, Bend, OR 97702, USA', '97702'],
            ['timberlinelodge', 'Timberline Lodge', 'Oregon', '27500 E Timberline Road, Government Camp, OR 97028, USA', '97028'],
            ['skihomewood', 'Homewood', 'California', '5145 W Lake Blvd, Homewood, CA 96141, USA', '96141'],
            ['eldoramtnresort', 'Eldora Mountain', 'Colorado', '2861 Eldora Ski Rd #140, Nederland, CO 80466, USA', '80466'],
            ['bearvalleymountainupdates', 'Bear Valley Mountain', 'California', '2280 state Rte 207, Bear Valley, CA 95223, USA', '95223'],
            ['diamondpeak', 'Diamond Peak', 'Nevada', '1210 Ski Way, Incline Village, NV 89451, USA', '89451'],
            ['49degreesnorth', '49 Degrees North', 'Washington', '3311 Flowery Trail Rd, Chewelah, WA 99109, USA', '99109'],
            ['lookoutpass', 'Lookout Pass', 'Idaho', 'I-90 Exit 0, Mullan, ID 83846, USA', '83846'],
            ['discoveryskiarea', 'Discovery Ski Area', 'Montana', '180 Discovery Basin Rd, Anaconda, MT 59711, USA', '59711'],
            ['pebbleskiarea', 'Pebble Creek', 'Idaho', '3340 E Green Canyon Rd, Inkom, ID 83245, USA', '83245'],
            ['sunlightmountainresort', 'Sunlight Mountain', 'Colorado', '10901 Co Rd 117, Glenwood Springs, CO 81601, USA', '81601'],
            ['skibowl', 'Mount Hood Skibowl', 'Oregon', '87000 U.S. 26, Government Camp, OR 97028, USA', '97028'],
            ['dodgeridge', 'Dodge Ridge', 'California', '1 Dodge Ridge Rd, Pinecrest, CA 95364, USA', '95364'],
            ['anthonylakes', 'Anthony Lakes Mountain', 'Oregon', '47500 Anthony Lakes Hwy, North Powder, OR 97867, USA', '97867']]

stats_table1 = []

for i in range(0, 25):
    
    # get the longitude and latitude of the location of the resort
    # sign up at the Google Cloud and get your own key for the Google API
    google_key = 'xxx' # enter your own Google API key here
    response = requests.get('https://maps.googleapis.com/maps/api/geocode/json?address=' + str(accounts[i][3]) + '&key=' + str(google_key))
    resp_json_payload = response.json()
    longitude = resp_json_payload['results'][0]['geometry']['location']['lng']
    latitude = resp_json_payload['results'][0]['geometry']['location']['lat']
    
    # set the URL to the Instagram account
    url = 'https://www.instagram.com/' + str(accounts[i][0])
    
    # make a folder to save the data
    path = urllib.parse.urlparse(url).path
    folder_name = path.replace('/', '')
    folder_path = 'images/{}/'.format(folder_name)
    folder_path_trash = 'images/{}/trash'.format(folder_name)
    
    account_data1 = pd.read_csv(os.path.join(folder_path, str(accounts[i][0]) + "_stats.csv"), sep = ',', encoding = "ISO-8859-1")
    account_data1 = account_data1[(account_data1['video_link'].isnull()) & (account_data1['image_link'].notnull())]
    number_input = account_data1.shape[0]
    
    account_data2 = pd.read_csv(os.path.join(folder_path, str(accounts[i][0]) + "_pictures.csv"), sep = ',', encoding = "ISO-8859-1")
    image_list = account_data2.values.tolist()
    
    # let's check if there are downloaded files without statistics (both in the useful files as well as in the trash folder)
    account_data3 = account_data1[['image_id']].values.tolist()
    account_data4 = [item for sublist in account_data3 for item in sublist]
    
    jpg_files = glob.glob1(folder_path, "*.jpg")
    
    if account_data1.values.tolist()[0][0] < 10:
        image_ids = [int(image_id[:6]) for image_id in jpg_files]
    else:
        image_ids = [int(image_id[:7]) for image_id in jpg_files]
        
    for k in range(0, len(image_ids)):
        if image_ids[k] not in account_data4:
            print('The picture with id (not in trash)', image_ids[k], 'has no statistics.')
    
    jpg_files = glob.glob1(folder_path_trash, "*.jpg")
    
    if account_data1.values.tolist()[0][0] < 10:
        image_ids = [int(image_id[:6]) for image_id in jpg_files]
    else:
        image_ids = [int(image_id[:7]) for image_id in jpg_files]
        
    for k in range(0, len(image_ids)):
        if image_ids[k] not in account_data4:
            print('The picture with id (in trash)', image_ids[k], 'has no statistics.')
    
    number_useful_images = len(glob.glob1(folder_path, "*.jpg"))
    number_images_trash = len(glob.glob1(folder_path + 'trash/', "*.jpg"))
    number_images_total = number_useful_images + number_images_trash
    
    account_list = [accounts[i][0], accounts[i][1], accounts[i][2], longitude, latitude, image_list[0][3], number_useful_images, number_images_trash, number_images_total, accounts[i][4]]
    stats_table1.append(account_list)
    
    print('The number of followers of', image_list[0][0],':', image_list[0][3])
    print('The number of images without video in the input file:', number_input)
    print('The number of useful scraped images:', number_useful_images)
    print('The number of scraped images in trash:', number_images_trash)
    print('The number of total scraped images:', number_images_total)
    print('The number of missing images:', number_input - number_images_total)

In [ ]:
stats_table2 = []
    
for i in range(25, 50):
     
    # get the longitude and latitude of the location of the resort
    google_key = 'xxx' # enter your own Google API key here
    response = requests.get('https://maps.googleapis.com/maps/api/geocode/json?address=' + str(accounts[i][3]) + '&key=' + str(google_key))
    resp_json_payload = response.json()
    longitude = resp_json_payload['results'][0]['geometry']['location']['lng']
    latitude = resp_json_payload['results'][0]['geometry']['location']['lat']
    
    # set the URL to the Instagram account
    url = 'https://www.instagram.com/' + str(accounts[i][0])
    
    # make a folder to save the data
    path = urllib.parse.urlparse(url).path
    folder_name = path.replace('/', '')
    folder_path = 'images/{}/'.format(folder_name)
    folder_path_trash = 'images/{}/trash'.format(folder_name)
    
    account_data1 = pd.read_csv(os.path.join(folder_path, str(accounts[i][0]) + "_stats.csv"), sep = ',', encoding = "ISO-8859-1")
    account_data1 = account_data1[(account_data1['video_link'].isnull()) & (account_data1['image_link'].notnull())]
    number_input = account_data1.shape[0]
    
    account_data2 = pd.read_csv(os.path.join(folder_path, str(accounts[i][0]) + "_pictures.csv"), sep = ',', encoding = "ISO-8859-1")
    image_list = account_data2.values.tolist()
    
    # let's check if there are downloaded files without statistics (both in the useful files as well as in the trash folder)
    account_data3 = account_data1[['image_id']].values.tolist()
    account_data4 = [item for sublist in account_data3 for item in sublist]
    
    jpg_files = glob.glob1(folder_path, "*.jpg")
    
    if account_data1.values.tolist()[0][0] < 10:
        image_ids = [int(image_id[:6]) for image_id in jpg_files]
    else:
        image_ids = [int(image_id[:7]) for image_id in jpg_files]
        
    for k in range(0, len(image_ids)):
        if image_ids[k] not in account_data4:
            print('The picture with id (not in trash)', image_ids[k], 'has no statistics.')
    
    jpg_files = glob.glob1(folder_path_trash, "*.jpg")
    
    if account_data1.values.tolist()[0][0] < 10:
        image_ids = [int(image_id[:6]) for image_id in jpg_files]
    else:
        image_ids = [int(image_id[:7]) for image_id in jpg_files]
        
    for k in range(0, len(image_ids)):
        if image_ids[k] not in account_data4:
            print('The picture with id (in trash)', image_ids[k], 'has no statistics.')
    
    number_useful_images = len(glob.glob1(folder_path, "*.jpg"))
    number_images_trash = len(glob.glob1(folder_path + 'trash/', "*.jpg"))
    number_images_total = number_useful_images + number_images_trash
    
    account_list = [accounts[i][0], accounts[i][1], accounts[i][2], longitude, latitude, image_list[0][3], number_useful_images, number_images_trash, number_images_total, accounts[i][4]]
    stats_table2.append(account_list)
    
    print('The number of followers of', image_list[0][0],':', image_list[0][3])
    print('The number of images without video in the input file:', number_input)
    print('The number of useful scraped images:', number_useful_images)
    print('The number of scraped images in trash:', number_images_trash)
    print('The number of total scraped images:', number_images_total)
    print('The number of missing images:', number_input - number_images_total)

In [ ]:
stats_table3 = []

for i in range(50, 75):
     
    # get the longitude and latitude of the location of the resort
    google_key = 'xxx' # enter your own Google API key here
    response = requests.get('https://maps.googleapis.com/maps/api/geocode/json?address=' + str(accounts[i][3]) + '&key=' + str(google_key))
    resp_json_payload = response.json()
    longitude = resp_json_payload['results'][0]['geometry']['location']['lng']
    latitude = resp_json_payload['results'][0]['geometry']['location']['lat']
    
    # set the URL to the Instagram account
    url = 'https://www.instagram.com/' + str(accounts[i][0])
    
    # make a folder to save the data
    path = urllib.parse.urlparse(url).path
    folder_name = path.replace('/', '')
    folder_path = 'images/{}/'.format(folder_name)
    folder_path_trash = 'images/{}/trash'.format(folder_name)
    
    account_data1 = pd.read_csv(os.path.join(folder_path, str(accounts[i][0]) + "_stats.csv"), sep = ',', encoding = "ISO-8859-1")
    account_data1 = account_data1[(account_data1['video_link'].isnull()) & (account_data1['image_link'].notnull())]
    number_input = account_data1.shape[0]
    
    account_data2 = pd.read_csv(os.path.join(folder_path, str(accounts[i][0]) + "_pictures.csv"), sep = ',', encoding = "ISO-8859-1")
    image_list = account_data2.values.tolist()
    
    # let's check if there are downloaded files without statistics (both in the useful files as well as in the trash folder)
    account_data3 = account_data1[['image_id']].values.tolist()
    account_data4 = [item for sublist in account_data3 for item in sublist]
    
    jpg_files = glob.glob1(folder_path, "*.jpg")
    
    if account_data1.values.tolist()[0][0] < 10:
        image_ids = [int(image_id[:6]) for image_id in jpg_files]
    else:
        image_ids = [int(image_id[:7]) for image_id in jpg_files]
        
    for k in range(0, len(image_ids)):
        if image_ids[k] not in account_data4:
            print('The picture with id (not in trash)', image_ids[k], 'has no statistics.')
    
    jpg_files = glob.glob1(folder_path_trash, "*.jpg")
    
    if account_data1.values.tolist()[0][0] < 10:
        image_ids = [int(image_id[:6]) for image_id in jpg_files]
    else:
        image_ids = [int(image_id[:7]) for image_id in jpg_files]
        
    for k in range(0, len(image_ids)):
        if image_ids[k] not in account_data4:
            print('The picture with id (in trash)', image_ids[k], 'has no statistics.')
    
    number_useful_images = len(glob.glob1(folder_path, "*.jpg"))
    number_images_trash = len(glob.glob1(folder_path + 'trash/', "*.jpg"))
    number_images_total = number_useful_images + number_images_trash
    
    account_list = [accounts[i][0], accounts[i][1], accounts[i][2], longitude, latitude, image_list[0][3], number_useful_images, number_images_trash, number_images_total, accounts[i][4]]
    stats_table3.append(account_list)
    
    print('The number of followers of', image_list[0][0],':', image_list[0][3])
    print('The number of images without video in the input file:', number_input)
    print('The number of useful scraped images:', number_useful_images)
    print('The number of scraped images in trash:', number_images_trash)
    print('The number of total scraped images:', number_images_total)
    print('The number of missing images:', number_input - number_images_total)

In [ ]:
stats_table4 = []

for i in range(75, len(accounts)):
     
    # get the longitude and latitude of the location of the resort
    google_key = 'xxx' # enter your own Google API key here
    response = requests.get('https://maps.googleapis.com/maps/api/geocode/json?address=' + str(accounts[i][3]) + '&key=' + str(google_key))
    resp_json_payload = response.json()
    longitude = resp_json_payload['results'][0]['geometry']['location']['lng']
    latitude = resp_json_payload['results'][0]['geometry']['location']['lat']
    
    # set the URL to the Instagram account
    url = 'https://www.instagram.com/' + str(accounts[i][0])
    
    # make a folder to save the data
    path = urllib.parse.urlparse(url).path
    folder_name = path.replace('/', '')
    folder_path = 'images/{}/'.format(folder_name)
    folder_path_trash = 'images/{}/trash'.format(folder_name)
    
    account_data1 = pd.read_csv(os.path.join(folder_path, str(accounts[i][0]) + "_stats.csv"), sep = ',', encoding = "ISO-8859-1")
    account_data1 = account_data1[(account_data1['video_link'].isnull()) & (account_data1['image_link'].notnull())]
    number_input = account_data1.shape[0]
    
    account_data2 = pd.read_csv(os.path.join(folder_path, str(accounts[i][0]) + "_pictures.csv"), sep = ',', encoding = "ISO-8859-1")
    image_list = account_data2.values.tolist()
    
    # let's check if there are downloaded files without statistics (both in the useful files as well as in the trash folder)
    account_data3 = account_data1[['image_id']].values.tolist()
    account_data4 = [item for sublist in account_data3 for item in sublist]
    
    jpg_files = glob.glob1(folder_path, "*.jpg")
    
    if account_data1.values.tolist()[0][0] < 10:
        image_ids = [int(image_id[:6]) for image_id in jpg_files]
    else:
        image_ids = [int(image_id[:7]) for image_id in jpg_files]
        
    for k in range(0, len(image_ids)):
        if image_ids[k] not in account_data4:
            print('The picture with id (not in trash)', image_ids[k], 'has no statistics.')
    
    jpg_files = glob.glob1(folder_path_trash, "*.jpg")
    
    if account_data1.values.tolist()[0][0] < 10:
        image_ids = [int(image_id[:6]) for image_id in jpg_files]
    else:
        image_ids = [int(image_id[:7]) for image_id in jpg_files]
        
    for k in range(0, len(image_ids)):
        if image_ids[k] not in account_data4:
            print('The picture with id (in trash)', image_ids[k], 'has no statistics.')
    
    number_useful_images = len(glob.glob1(folder_path, "*.jpg"))
    number_images_trash = len(glob.glob1(folder_path + 'trash/', "*.jpg"))
    number_images_total = number_useful_images + number_images_trash
    
    account_list = [accounts[i][0], accounts[i][1], accounts[i][2], longitude, latitude, image_list[0][3], number_useful_images, number_images_trash, number_images_total, accounts[i][4]]
    stats_table4.append(account_list)
    
    print('The number of followers of', image_list[0][0],':', image_list[0][3])
    print('The number of images without video in the input file:', number_input)
    print('The number of useful scraped images:', number_useful_images)
    print('The number of scraped images in trash:', number_images_trash)
    print('The number of total scraped images:', number_images_total)
    print('The number of missing images:', number_input - number_images_total)

In [ ]:
stats_table = stats_table1 + stats_table2 + stats_table3 + stats_table4

In [ ]:
# see a preview of the resulting table
stats_table[0]

In [ ]:
# calculate the total number of scraped pictures
print('The total number of useful images:', sum([i[6] for i in stats_table]))
print('The total number of images in trash:', sum([i[7] for i in stats_table]))
print('The total number of images:', sum([i[8] for i in stats_table]))

In [ ]:
# print this table to a csv
df = pd.DataFrame(stats_table)
df.columns = ['accountname', 'resort', 'state', 'longitude', 'latitude', 'followers', 'number_useful_pictures', 'number_images_trash', 'number_images_total', 'zip_code']
df.to_csv('results/stats_table.csv', index = False)